In [55]:
import glob

from src.create_annotations import *

# Label ids of the dataset
category_ids = {
    "rebar": 0,
    "spall": 1,
    "crack": 2,
}

# Define which colors match which categories in the images
category_colors = {
    "(255, 0, 0)": 0, # "rebar"
    "(0, 255, 0)": 1, # "spall"
    "(0, 0, 255)": 2, # "crack"
}

# Define the ids that are a multiplolygon. In our case: wall, roof and sky
# multipolygon_ids = [ 0, 1, 2]
multipolygon_ids = [5]

# Get "images" and "annotations" info 
def images_annotations_info(root_path):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []

    for main_image in glob.glob(root_path + "images/"+ "*.jpg"):
        main_img = Image.open(main_image)
        width, height = main_img.size
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        filename = os.path.basename(main_image).split(".")[0]
        # filename = os.path.splitext(file)[0]
        rebarmask_file = os.path.join(root_path, "masks",filename + 'rebar' + '.jpg')
        spallmask_file = os.path.join(root_path, "masks", filename + 'spall' + '.jpg')
        crackmask_file = os.path.join(root_path, "masks", filename + 'crack' + '.jpg')
        rebarmask = Image.new( mode = "RGB", size = main_img.size, color = (0, 0, 0))
        spallmask = Image.new( mode = "RGB", size = main_img.size, color = (0, 0, 0))
        crackmask = Image.new( mode = "RGB", size = main_img.size, color = (0, 0, 0))
        # Open the image and (to be sure) we convert it to RGB
        if os.path.exists(rebarmask_file):
            color1  = Image.new( mode = "RGB", size = main_img.size, color = (255, 0, 0) )
            mask1 = Image.open(rebarmask_file)#.convert("RGB")
            rebarmask = Image.composite(color1,color1,mask1)
        if os.path.exists(spallmask_file):
            color2  = Image.new( mode = "RGB", size = main_img.size, color = (0, 255, 0) )
            mask2 = Image.open(spallmask_file)#.convert("RGB")
            spallmask = Image.composite(color2,color2,mask2)
        if os.path.exists(crackmask_file):
            color3  = Image.new( mode = "RGB", size = main_img.size, color = (0, 0, 255) )
            mask3 = Image.open(crackmask_file)#.convert("RGB")
            crackmask = Image.composite(color3,color3,mask3)
        # mask_image_open = Image.merge("RGB",(rebarmask,spallmask,crackmask))
        # mask_image_open = Image.merge(tmp,crackmask)
        pix = np.array(rebarmask)+np.array(spallmask)+np.array(crackmask)
        mask_image_open = Image.fromarray(pix)
        # "images" info
        image = create_image_annotation(filename, width, height, image_id)
        images.append(image)


        sub_masks = create_sub_masks(mask_image_open, width, height)
        for color, sub_mask in sub_masks.items():
            if color in category_colors.keys():
                category_id = category_colors[color]
            else:
                res = eval(color)
                category_id = res.index(255)
            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)
            # print(segmentations)
            # Check if we have classes that are a multipolygon
            if category_id in multipolygon_ids:
                # Combine the polygons to calculate the bounding box and area
                multi_poly = MultiPolygon(polygons)
                print("multi_poly.bounds",multi_poly.bounds)
                annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

                annotations.append(annotation)
                annotation_id += 1
            else:
                # print(len(polygons))
                for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, category_id, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id


In [56]:
if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()
    root = "../../data/"
    
    for keyword in ["train"]:
        root_path = root+"dataset/"
        mask_path = root_path + "masks"
        
        # Create category section
        coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(root_path)

        with open("output/{}.json".format(keyword),"w") as outfile:
            json.dump(coco_format, outfile)
        
        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))